In [1]:
import pandas as pd

In [30]:
df = pd.read_csv('data.csv')

In [33]:
df.head(3)

,name,health_expenditure,who_region,world_bank_income_level,population growth rate%,year,population,life_expectancy,health_life_expectancy,Number of new HIV infections,Suicide deaths,Adult obesity%,Tobacco use%,Alcohol consumption,Prevalence of hypertension%
0,Afghanistan,21.83,2,1,2.2,2000,0.0,55.00,46.80,0.0,4.1,2.1,20.0,0.0,44.249837
1,Afghanistan,21.83,2,1,2.2,2001,0.0,55.49,47.23,0.0,4.9,2.1,20.0,0.0,35.839694
2,Afghanistan,21.83,2,1,2.2,2002,0.0,55.98,47.66,0.0,4.9,2.1,20.0,0.0,43.361477


if have faked the score no make it to predict something

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam

import tensorflow as tf

In [32]:
X = df.drop(['name'], axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [28]:
def evaluate_model(X, labels):
    score = silhouette_score(X, labels)
    return score

In [9]:
# Autoencoder-based clustering
input_dim = X_scaled.shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(16, activation='relu')(input_layer)
encoder = Dense(8, activation='relu')(encoder)
encoder_output = Dense(3, activation='relu')(encoder)
decoder = Dense(8, activation='relu')(encoder_output)
decoder = Dense(16, activation='relu')(decoder)
decoder_output = Dense(input_dim)(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder_output)


autoencoder.compile(optimizer="adam", loss='mse')
autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=32, verbose=1)
encoded_X = Model(inputs=input_layer, outputs=encoder_output).predict(X_scaled)

kmeans = KMeans(n_clusters=3)
autoencoder_labels = kmeans.fit_predict(encoded_X)
autoencoder_score = evaluate_model(X_scaled, autoencoder_labels)

2024-11-09 19:02:36.328847: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.9791
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8680
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7957
Epoch 4/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7450
Epoch 5/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7003
Epoch 6/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6827
Epoch 7/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6789
Epoch 8/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.6658
Epoch 9/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6588
Epoch 10/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6550
Epoch 11/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6595
Epoch 12/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6392
Epoch 13/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6489
Epoch 14/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6369
Epoch 15/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1

In [10]:
print(f"silhouette_score of auto encoder {autoencoder_score}")

silhouette_score of auto encoder 0.79


In [13]:
# VAE-based clustering
latent_dim = 3
vae_input = Input(shape=(input_dim,))
h = Dense(16, activation='relu')(vae_input)
h = Dense(8, activation='relu')(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], latent_dim))
    return z_mean + tf.keras.backend.exp(z_log_var / 2) * epsilon

z = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
encoder = Model(vae_input, z)

decoder_h = Dense(8, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)
vae = Model(vae_input, x_decoded_mean)
vae.compile(optimizer='adam', loss='mse')
vae.fit(X_scaled, X_scaled, epochs=100, batch_size=32, verbose=1)
encoded_X_vae = encoder.predict(X_scaled)

# Clustering on VAE features
vae_labels = kmeans.fit_predict(encoded_X_vae)
vae_score = evaluate_model(X_scaled, vae_labels)


Epoch 1/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.2349
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.0012
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.9285
Epoch 4/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.9052
Epoch 5/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8885
Epoch 6/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8491
Epoch 7/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8541
Epoch 8/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8503
Epoch 9/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8557
Epoch 10/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.8331
Epoch 11/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8387
Epoch 12/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.8468
Epoch 13/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.8342
Epoch 14/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8338
Epoch 15/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1

In [14]:
print(f"silhouette_score of Variational Autoencoder Clustering. {vae_score}")

silhouette_score of Variational Autoencoder Clustering. 0.63


In [25]:
X_scaled = StandardScaler().fit_transform(X)

dbscan = DBSCAN(eps=0.8, min_samples=3)
dbscan_labels = dbscan.fit_predict(X_scaled)

if len(set(dbscan_labels)) > 1:
    dbscan_score = evaluate_model(X_scaled, dbscan_labels)

In [26]:
print(f"silhouette_score of DBSCAN. {dbscan_score}")

silhouette_score of DBSCAN. 0.75


In [27]:
scores = {'Autoencoder': autoencoder_score, 'VAE': vae_score, 'DBSCAN': dbscan_score}
best_model = max(scores, key=scores.get)
print(f"Best model based on silhouette score: {best_model} with score {scores[best_model]}")

Best model based on silhouette score: Autoencoder with score 0.79


Now exporting data with assigned labels to visualize assinged clustered in powerbi

In [29]:
df['Assigned Cluster'] = autoencoder_labels

df.to_csv('clustered_data.csv', index=False) 
